In [1]:
%matplotlib inline
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import itertools as it

<b>Problem 1</b>: For a 3 $\times$ 3 lattice, there are $2^9$ configurations possible.

In [2]:
configs = 2**9
print(configs)
#print(np.permutation(9,2))

512


In [3]:
def is_in(item, array):
    """
    This function returns true if the given item is in the
    array and false if the given item is not in the array.
    """
    value = False
    ctr = 0
    while((value is False) and (ctr < array.shape[-1])):
        if (array[ctr] == item):
            value = True
        ctr += 1
    return value

In [4]:
a = "1111"
print([char for char in a])

['1', '1', '1', '1']


In [5]:
def list_to_array(some_list, dim):
    length = len(some_list)
    temp = []
    for i in some_list:
        i.split()  
    array = np.zeros()

In [6]:
# testing my function that tests if an element is in an array
A = np.array([1, 2, 3, 4, 5, 6])
print(is_in(8, A))

False


In [7]:
def create_lattice(dim=3):
    n2 = dim * dim
    configs = 2**n2
    spins = ['1', '0']
    possible_configs = ["".join(item) for item in it.product(spins, repeat=n2)]
    spin_tensor = np.zeros([dim, dim, configs], float)
    for k in range(configs):
        config_temp = possible_configs[k] 
        temp = [char for char in config_temp]
        array_temp = np.array(temp, float)
        reshaped = np.reshape(array_temp, (3, 3))
        spin_tensor[:, :, k] = np.where(reshaped == 0, -1, reshaped)
    return spin_tensor

[[[ 1.  1.  1. ... -1. -1. -1.]
  [ 1.  1.  1. ... -1. -1. -1.]
  [ 1.  1.  1. ... -1. -1. -1.]]

 [[ 1.  1.  1. ... -1. -1. -1.]
  [ 1.  1.  1. ... -1. -1. -1.]
  [ 1.  1.  1. ... -1. -1. -1.]]

 [[ 1.  1.  1. ... -1. -1. -1.]
  [ 1.  1. -1. ...  1. -1. -1.]
  [ 1. -1.  1. ... -1.  1. -1.]]]
